# Ubuntu 환경에서 terraform-jenkins-git 환경 구축 on GCP

### 목차
### 1. WSL 및 Ubuntu설치
### 2. gcloud sdk 설치 on Ubuntu
### 3. VM instance 생성
### 4. Jenkins 설치
### 5. terraform 설치
### 6. git 설치
### 7.  jenkins <-> git 연동
### 8. jenkins <-> terraform 연동
### 9. 젠킨스 테스트

---
---
## 1. WSL 및 Ubuntu설치

```
1. Microsoft store에서 windows terminal 설치
2. windows powershell 관리자 권한으로 실행
wsl --install
3. ubuntu 설치
wsl --install -d Ubuntu-22.04
4. ubuntu 실행
```

----
## 2. gcloud sdk 설치 on Ubuntu
```
1. gcloud sdk 설치 위해 python3 설치
sudo apt update
sudo apt upgrade
sudo apt install -y python3 python3-pip
2. gcloud sdk 설치
curl https://sdk.cloud.google.com | bash
exec -l $SHELL
gcloud version
3. gcp 연동
gcloud init
gcloud cloud-shell ssh
gcloud auth login
```

---
## 3. VM instance 생성
```
1. 인스턴스 api enable 
gcloud auth login
gcloud services enable compute.googleapis.com

2. gcloud 명령어로 인스턴스 생성(zone : asia-northeast3-a)
gcloud compute instances create jenkins \
--machine-type e2-medium
3. 인스턴스 목록 확인
gcloud compute instances list
4. 인스턴스 http,https 방화벽 태그 추가
gcloud compute instances add-tags jenkins \
--tags http-server,https-server
5. 8080 포트 방화벽 룰 생성
gcloud compute firewall-rules create jenkins-rull \
--allow=tcp:8080 \
--description="Allow incoming traffic on TCP port 8080"\
--direction=INGRESS
6. 8080 포트 방화벽 룰 추가
gcloud compute instances add-tags jenkins \
--tags jekins-rull
7. 인스턴스 ssh로 접속
gcloud compute ssh thsehdrl94@jenkins
sudo apt-get update
sudo apt-get upgrade

```

---
## 4. Jenkins 설치
```
1. jenkins 설치
- 자바설치 
sudo apt install openjdk-17-jre
- 젠킨스 설치
curl -fsSL https://pkg.jenkins.io/debian/jenkins.io-2023.key | sudo tee \
  /usr/share/keyrings/jenkins-keyring.asc > /dev/null
echo deb [signed-by=/usr/share/keyrings/jenkins-keyring.asc] \
  https://pkg.jenkins.io/debian binary/ | sudo tee \
  /etc/apt/sources.list.d/jenkins.list > /dev/null
sudo apt-get update
sudo apt-get install jenkins

- 젠킨스 실행
sudo systemctl enable jenkins
sudo systemctl start jenkins
```

-  젠킨스 접속
http://[EXTERNAL IP]:8080/

---
## 5. terraform 설치
```
1. terraform 패키지
sudo apt-get update && sudo apt-get install -y gnupg software-properties-common curl
2. HashiCorp 의 GPG Key 등록
curl -fsSL https://apt.releases.hashicorp.com/gpg | sudo apt-key add -
3.. HashiCorp의 Linux Repository 등록
sudo apt-add-repository "deb [arch=amd64] https://apt.releases.hashicorp.com $(lsb_release -cs) main"
4. terraform 설치
sudo apt-get update && sudo apt-get install terraform
5. terraform 버전확인
terraform version

```
---
## 6. git 설치
```
1. git 설치
sudo apt install git
git config --global user.email "thsehdrl94@gmail.com"
git config --global user.name "laits1"
- git push 인증 반복해제
git config --global credential.helper store

```
---

## 7.  jenkins <-> git 연동

```
13. jenkins <-> git 연동
git 홈페이지-> 우상단 프로필 -> settings -> Developer settings -> Personal access tokens -> Generate new token
repo, admin:repo_hook 체크후 생성
키 복사

- github - settings - webhooks - add webhook
payload URL : http://[EXTERNAL IP]:8080/github-webhook/
content type : application/json

- Jenkins 관리 -> System -> GitHub Servers
name : jenkins-server
credential add ->
kind : Secret text
secret : git token key
id : git id
manage hook 체크


새로운 ITEM 
pipline
- GitHub hook trigger for GITScm polling 체크
- github project 체크 -> 깃주소입력
- pipline -> definiton : pipeline script from SCM
  SCM : git
  -> Repository URL 입력
  Credential
- Script Path
 Jenkinsfile
  




# 권한 문제일시jenkins 권한 추가
sudo visudo에서
jenkins ALL=(ALL) NOPASSWD: ALL


# Jenkins pipline 파일 생성


```

---
## 8. jenkins <-> terraform 연동
```
1. jenkins 플러그인 - terraform 설치

2. IAM 구성
- gcloud init 하고 gcloud 계정 접속
- 서비스 계정 생성
gcloud iam service-accounts create jenkins --display-name jenkins
- 서비스 계정 이메일 주소 및 현재 Google Cloud 프로젝트 ID를 환경 변수에 저장
export SA_EMAIL=$(gcloud iam service-accounts list \
    --filter="displayName:jenkins" --format='value(email)')
export PROJECT=$(gcloud info --format='value(config.project)')
- 서비스 계정에 역할 부여
gcloud projects add-iam-policy-binding $PROJECT \
    --role roles/storage.admin \
    --member serviceAccount:$SA_EMAIL
gcloud projects add-iam-policy-binding $PROJECT \
    --role roles/compute.instanceAdmin.v1 \
    --member serviceAccount:$SA_EMAIL
gcloud projects add-iam-policy-binding $PROJECT \
    --role roles/compute.networkAdmin \
    --member serviceAccount:$SA_EMAIL
gcloud projects add-iam-policy-binding $PROJECT \
    --role roles/compute.securityAdmin \
    --member serviceAccount:$SA_EMAIL
gcloud projects add-iam-policy-binding $PROJECT \
    --role roles/iam.serviceAccountActor \
    --member serviceAccount:$SA_EMAIL
    
    
gcloud projects add-iam-policy-binding $PROJECT \
    --member serviceAccount:$SA_EMAIL \
    --role roles/compute.instanceAdmin
   
- 서비스 계정 키 파일 생성
gcloud iam service-accounts keys create jenkins-sa.json --iam-account $SA_EMAIL


- main.tf 생성

```
---

## 9. 젠킨스 테스트
```
git push test
```

In [ ]:
# Jenkinsfile
pipeline {
  agent any

  environment {
    GIT_URL = "https://github.com/laits1/test-project1.git"
  }

  stages {
    stage('Pull') {
      steps {
        git(url: "${GIT_URL}", branch: "master", changelog: true, poll: true)
      }
    }
    stage('Terraform Init') {
            steps {
                sh 'terraform init'
            }
        }
        stage('Terraform Plan') {
            steps {
                sh 'terraform plan'
            }
        }
        stage('Terraform Apply') {
            steps {
                sh 'terraform apply -auto-approve'
            }
        }
  }
}


In [ ]:
# main.tf

provider "google" {
  credentials = file("/home/thsehdrl94/test/jenkins-sa.json")
  project     = "test-project1-394523"
  region      = "asia-northeast3"  # 원하는 리전으로 변경
}

resource "google_compute_instance" "default" {
  name         = "terraform"
  machine_type = "n1-standard-1"
  zone         = "asia-northeast3-a"  # 원하는 존으로 변경

  boot_disk {
    initialize_params {
      image = "debian-cloud/debian-10"  # 사용할 이미지
    }
  }

  network_interface {
    network = "default"
  }
}